In [2]:
import cv2
import tensorflow as tf
import numpy as np
from sklearn.model_selection import train_test_split
import tf2onnx
import pickle

2023-11-15 14:56:47.268795: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-15 14:56:48.099233: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/asif/.local/lib/python3.7/site-packages/cv2/../../lib64:
2023-11-15 14:56:48.099481: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-11-15 14:56:51.805015: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror:

In [3]:
dataset = tf.keras.utils.image_dataset_from_directory("Dataset/", batch_size=512, image_size=(120,120), shuffle=False)

Found 512 files belonging to 2 classes.


2023-11-15 14:57:01.711457: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/asif/.local/lib/python3.7/site-packages/cv2/../../lib64:
2023-11-15 14:57:01.712955: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2023-11-15 14:57:01.713606: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (ubuntu-asif): /proc/driver/nvidia/version does not exist
2023-11-15 14:57:01.719455: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compile

In [4]:
x = np.asarray(list(dataset.as_numpy_iterator())[0][0])/255
y = np.asarray(tf.one_hot(list(dataset.as_numpy_iterator())[0][1],depth=2))

In [5]:
x_train, x_test, y_train, y_test = train_test_split(x,y,train_size=0.7,shuffle=True)

In [6]:
M1 = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(48, 6, strides=(4, 4), input_shape=(120, 120, 3)),
    tf.keras.layers.MaxPool2D(pool_size=(3, 3), strides=(2, 2)),
    tf.keras.layers.Conv2D(64, 5, padding='same'),
    tf.keras.layers.MaxPool2D(pool_size=(3, 3), strides=(2, 2)),
    tf.keras.layers.Conv2D(64, 3, padding='same'),
    tf.keras.layers.Conv2D(64, 3, padding='same'),
    tf.keras.layers.Conv2D(64, 3, padding='same'),
    tf.keras.layers.MaxPool2D(pool_size=(3, 3), strides=(2, 2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, 'relu'),
    tf.keras.layers.Dense(64, 'relu'),
    tf.keras.layers.Dense(2, 'sigmoid')
])

In [7]:
M1.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [8]:
M1.fit(x_train, y_train, epochs=8)

Epoch 1/8
12/12 [==============================] - 7s 233ms/step - loss: 0.4822 - accuracy: 0.7598
Epoch 2/8
12/12 [==============================] - 2s 207ms/step - loss: 0.1701 - accuracy: 0.9302
Epoch 3/8
12/12 [==============================] - 2s 198ms/step - loss: 0.0217 - accuracy: 0.9944
Epoch 4/8
12/12 [==============================] - 2s 205ms/step - loss: 0.0159 - accuracy: 0.9944
Epoch 5/8
12/12 [==============================] - 2s 204ms/step - loss: 0.0051 - accuracy: 0.9972
Epoch 6/8
12/12 [==============================] - 3s 211ms/step - loss: 4.3469e-04 - accuracy: 1.0000
Epoch 7/8
12/12 [==============================] - 3s 213ms/step - loss: 6.5260e-04 - accuracy: 1.0000
Epoch 8/8
12/12 [==============================] - 2s 203ms/step - loss: 3.7696e-05 - accuracy: 1.0000


In [9]:
M1.evaluate(x_test, y_test)

5/5 [==============================] - 1s 70ms/step - loss: 1.3343e-04 - accuracy: 1.0000


[0.0001334347907686606, 1.0]

In [10]:
with open("dset.pickle", "wb") as f:
    pickle.dump((x, y), f)

In [13]:
spec = (tf.TensorSpec((None, 120, 120, 3), tf.float32, name="input"),)
output_path = "M1.onnx"
model_proto, _ = tf2onnx.convert.from_keras(M1, input_signature=spec, opset=13, output_path=output_path)

2023-11-15 15:03:01.909552: I tensorflow/core/grappler/clusters/single_machine.cc:358] Starting new session
2023-11-15 15:03:02.355174: I tensorflow/core/grappler/clusters/single_machine.cc:358] Starting new session
